In [ ]:
import pandas as pd

# === 1. Cargar tabla de verdad corregida ===
tabla = pd.read_excel("tabla_verdad_SPyNoticias_ Corregida.xlsx")

# === 2. Diccionario de palabras ===
diccionario = {
    "politico": {
        "positivo": ["acuerdo", "estabilidad", "cooperación", "democracia"],
        "negativo": ["crisis", "corrupción", "protesta", "conflicto"]
    },
    "cultural": {
        "positivo": ["arte", "innovación", "educación", "tolerancia"],
        "negativo": ["censura", "discriminación", "abandono", "crítica"]
    },
    "social": {
        "positivo": ["empleo", "igualdad", "salud", "bienestar"],
        "negativo": ["pobreza", "violencia", "inseguridad", "desempleo"]
    }
}

# === 3. Función para calcular valores numéricos de p, q, s ===
def calcular_valores(texto, dic):
    texto = texto.lower()
    def contar(sentimiento):
        return sum(1 for palabra in dic[sentimiento]["positivo"] if palabra in texto), \
               sum(1 for palabra in dic[sentimiento]["negativo"] if palabra in texto)
    
    resultados = {}
    for tema in dic.keys():
        pos, neg = contar(tema)
        total = pos + neg if (pos + neg) > 0 else 1
        score = (pos - neg) / total  # entre -1 y +1
        resultados[tema] = max(0, score)  # recorta negativos a 0
    return resultados["politico"], resultados["cultural"], resultados["social"]

# === 4. Evaluar el texto de entrada ===
texto_noticias = """
El nuevo acuerdo comercial entre países fortalece la cooperación política
y mejora la estabilidad social y cultural mediante la promoción de la educación.
"""

p_val, q_val, s_val = calcular_valores(texto_noticias, diccionario)
print(f"Valores -> p: {p_val:.2f}, q: {q_val:.2f}, s: {s_val:.2f}")

# === 5. Buscar la fila de la tabla correspondiente ===
tabla_match = tabla[(tabla['p'] == int(p_val > 0)) &
                    (tabla['q'] == int(q_val > 0)) &
                    (tabla['s'] == int(s_val > 0))]

# Si hay varias filas, tomamos la primera
fila = tabla_match.iloc[0]

# === 6. Determinar si la inferencia es alzista o bajista ===
# (la columna con valor 1 indica escenario alzista)
es_alzista = any(fila[3:] == 1)  # ignora las primeras 3 columnas (p, q, s)

# === 7. Calcular la tasa ===
tasa0 = 1.0  # ejemplo: tasa base del S&P500 normalizada
suma_valores = p_val + q_val + s_val

if es_alzista:
    tasa = tasa0 * (1 + suma_valores)
else:
    tasa = tasa0 * (1 - suma_valores)

print(f"\nInferencia: {'ALZISTA' if es_alzista else 'BAJISTA'}")
print(f"Tasa calculada: {tasa:.3f}")